# VDS 1022/i Oscilloscope

To install from this folder :

`python -m pip install .`

Help:

In [ ]:
from vds1022 import *
help(vds1022)

In [ ]:
from vds1022 import *
help(vds1022.Frames)

In [ ]:
from vds1022 import *
help(vds1022.Frame)

Note that the first call of `VDS1022()` will take a few seconds since it has to load the FPGA firmware.  
The device will remain connected unless `.dispose()` is called or if the kernel is restarted.  
Interrupting the kernel (`I`,`I`) will not terminate the connection.

### Examples :
1. [Measure voltage](#1.-Read-voltage)
2. [Acquire a signal](#2.-Acquire-a-signal)
3. [Live plotting](#3.-Live-plotting)
4. [Data logging](#4.-Data-logging)
5. [Continuous sampling](#5.-Continuous-sampling)
6. [Decoding](#6.-Decoding)
7. [Spectrum analysis](#7.-Spectrum-analysis)
8. [Calibration](#8.-Calibration)
9. [Release device](#9.-Release-device)

## 1. Measure voltage

Read RMS voltage on channel 1  
Available metrics: `size, rms, avg, max, min, std, levels, freq, phase, median, percentile`

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_channel(CH1, range='10v', offset=1/10, probe='x10')
frames = dev.fetch()
print("%s Vrms" % frames.ch1.rms())

Plotting

In [ ]:
frames.plot()

Measure in a loop (interrupt to stop)

In [ ]:
try:
    for frames in dev.fetch_iter(freq=2, autorange=False):
        print('CH1: %sv     ' % frames.ch1.rms(), end='\r')
except KeyboardInterrupt: pass

## 2. Acquire a signal

Autoset the device and plot the signals

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
frames = dev.autoset().fetch()
frames.plot()

Generate descriptive statistics.

In [ ]:
frames.describe()

Convert to [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe)

In [ ]:
frames.to_dataframe()

Convert to [Numpy ndarray](https://numpy.org/doc/stable/reference/arrays.ndarray.html).  

In [ ]:
frames.to_numpy()

Set a trigger and plot the signal.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_timerange('10ms')
dev.set_channel(CH1, range='20v', offset=5/10, probe='x10')
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='2v', position=1/2)
frames = dev.fetch()
frames.plot()

## 3. Live plotting


Plot the signals with AUTO auto sweep mode

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_sampling('250k')
dev.set_channel(CH1, range='20v', offset=5/10, probe='x10')
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='1v', position=1/2, sweep=AUTO)
dev.plot(freq=2);

In [ ]:
dev.stop();

Plot the signals with NORMAL sweep mode

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_sampling('250k')
dev.set_channel(CH1, range='20v', offset=5/10, probe='x10')
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='1v', position=1/2, sweep=NORMAL)
dev.plot(freq=2);

In [ ]:
dev.stop();

## 4. Data logging
This section provides examples to measure a signal at a defined interval.

Stream the RMS voltage to a plot every second.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_channel(CH1, range='10v', offset=1/10, probe='x10')
dev.stream(freq=2).rms().plot();

In [ ]:
dev.stop();

Stream the average voltage to stdout every second.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_channel(CH1, range='20v', offset=1/10, probe='x10')
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')
dev.stream(freq=1).avg().sink(print);

In [ ]:
dev.stop();

Device roll/slow mode

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_timerange('60s', roll=True)
dev.set_channel(CH1, range='10v', offset=1/10, probe='x10')
dev.plot(freq=1);

In [ ]:
dev.stop();

Help:

In [ ]:
from vds1022 import *
help(vds1022.Stream)

## 5. Continuous sampling

This section provides examples to aquire and plot continuous samples without interruption for a defined duration.  
While this device can acquire sampling frames at 100Ms/s, the maximum continuous sampling rate is around 100Ks/s.  
If the amount of collected data is too consequent, the ploting will have to go through rasterization.

Acquire continuously 10 samples per period for a 1Khz signal and plot :

In [2]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_sampling('25k')  # 1K samples per seconds * 10 samples
dev.set_channel(CH1, range='5v', offset=0, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='0.5v')

frames = dev.read('5s')
frames.plot()
print("Samples: %s" % frames.ch1.size)

Samples: 125680


In [ ]:
frames.slice('0.5s', '0.6s').plot()

Acquire continuously 10 samples per period for a 1Khz signal and plot with rasterisation :

In [ ]:
from vds1022 import *
import hvplot.pandas

dev = VDS1022(debug=0)
dev.set_sampling('10k')  # 1K samples per seconds * 10 samples
dev.set_channel(CH1, range='10v', offset=2/10, probe='x10')
dev.set_trigger(CH1, EDGE, FALL, level='2.5v')

df = dev.read('1s').to_dataframe()
df.hvplot(rasterize=True).opts(width=800, ylim=dev.ylim(), colorbar=False)

## 6. Decoding

TTL

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_sampling('10k')  # 1K samples per seconds * 10 samples
dev.set_channel(CH1, range='10v', offset=2/10, probe='x10')
dev.set_trigger(CH1, EDGE, FALL, level='2.5v')

frames = dev.read('1s')
frames.ch1.to_ttl()

I2C

In [ ]:
from vds1022 import *

dev = VDS1022()
dev.set_sampling('10k')  # signal frequency * n samples
dev.set_channel(CH1, range='20v', offset=5/10, probe='x10')  # sda
dev.set_channel(CH2, range='20v', offset=1/10, probe='x10')  # scl
dev.set_trigger(CH1, EDGE, FALL, position=1/20, level='1.2v')
frames = dev.fetch()
frames.plot()

for msg in frames.decode_i2c():
    print(msg, msg.data)

## 7. Spectrum analysis

Plot spectrum

In [ ]:
import matplotlib.pyplot as plt
from numpy import log10, arange, pi, sqrt
from vds1022 import *

# Setup device for 1kHz signal
dev = VDS1022(debug=0)
dev.set_sampling('50k')  # samples per second
dev.set_channel(CH1, range='20v', probe='x10', coupling=DC)
# dev.set_trigger(CH1)  # align phase angle origine at center

# Acquire samples and compute the discrete Fourier Transform
frame = dev.fetch().ch1
f, m, p = frame.spectrum()  # frequencies(Hz), magnitudes(Vmax), phases(-1+1)

# Plot
fig, axs = plt.subplots(4, 1, figsize=(10, 4 * 1.6), constrained_layout=True)
for ax in axs: ax.grid()
axs[0].set(ylabel='y(t)', xlabel='Time [ms]', ylim=frame.ylim)
axs[1].set(ylabel='Magnitude [Vrms]')
axs[2].set(ylabel='Magnitude [dBV]', yticks=arange(-80,20,20))
axs[3].set(ylabel='Phase -1+1', xlabel='Frequency [kHz]', ylim=(-1,1))
axs[0].plot(frame.x() * 1000, frame.y())
axs[1].plot(f / 1000, m / sqrt(2))         # Vmax to Vrms
axs[2].plot(f / 1000, 20 * log10(m))       # Vmax to log
axs[3].plot(f / 1000, p * (m > frame.sy))  # Phases -1+1
plt.show()

Extract components from signal.


In [ ]:
for f, m, p in frame.components(threshold='3%'):
    print("%6s Hz  %5.2f Vrms  %5.2f rad" % (f, m / sqrt(2), p * pi))

**Power Spectral Density (PSD)**

In [ ]:
import matplotlib.pyplot as plt
from vds1022 import *

# Setup device
dev = VDS1022(debug=0)
dev.set_sampling('100k')  #  1kHz * 100 samples
dev.set_channel(CH1, range='20v', probe='x10', coupling=DC)

# Acquire samples
frame = dev.fetch().ch1
xs, ys = frame.xy()

# Plot
fig, axs = plt.subplots(2, 1, figsize=(10, 4), constrained_layout=True)
for ax in axs: ax.grid()
axs[0].set(ylabel='y(t)', xlabel='Time [ms]', ylim=frame.ylim)
axs[0].plot(xs, ys)
axs[1].psd(ys, 512, 1 / frame.sx)
plt.show()

## 8. Frequency response

**Manually sweep frequencies and plot a Bode graph**  
The plotting is done once the excution is interrupted (Ctrl+C or I+I).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_sampling('10k')
dev.set_channel(CH1, range='10v', probe='x10')
dev.set_channel(CH2, range='10v', probe='x10')

points, freq_prev = { }, -1
try:
    for ch1, ch2 in dev.fetch_iter(freq=4):
        freq, phase = ch1.freq()  # compute frequency and phase
        freq = freq and round(freq)
        if freq and freq == freq_prev:
            points[freq] = 20 * np.log10(ch2.rms() / ch1.rms())
            dev.set_sampling(freq * 50)  # increase frequency
            print('%d: %dHz  %.fdBv   ' % (len(points), freq, points[freq]), end='\r')
        freq_prev = freq

except KeyboardInterrupt:
    x, y = zip(*sorted(points.items()))  # sort, unzip

    plt.subplots(figsize=(12, 4))
    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Gain [dBV]')
    plt.grid(True, which="both", axis='x')
    plt.grid(True, which="major", axis='y')
    plt.minorticks_on()
    plt.xscale('log')
    plt.xlim(10 ** int(np.log10(x[0])), 10 ** int(np.log10(x[-1] * 10)))
    plt.ylim(-60, 20)
    plt.plot(x, y)
    plt.show();

## 9. Calibration
Wait a few minutes for the temperature of the device to stabilize before running the calibration.  
Once done, the calibration is saved to a file which will be used from now on.  
To restore the factory calibration, simply delete the file.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.calibrate()

## 10. Release device

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.dispose()

## 11. Audio generator

This section provides examples to generate a signal with a sound card.  
For an accurate output, disable the sound effects in the sound card settings.  
The generated signal is arround +1,-1 volt depending on the volume.   

https://pyacq.readthedocs.io/en/latest/examples/pyaudio_oscope_local.html  
https://www.szynalski.com/tone-generator/  

**List devices**

In [ ]:
from vds1022.generator import Generator
Generator.print_devices()

**Play a indefinitely a sine wave on the default device.**

In [ ]:
from vds1022.generator import Generator

sg = Generator(device=None, scale=1)
sg.sine(freq=900)  # next channel (left)
sg.sine(freq=300)  # next channel (right)
sg.plot()
sg.play()

In [ ]:
sg.stop()

**Generate a composite signal**

In [ ]:
from vds1022.generator import Generator

sg = Generator(device=None, scale=0.5)

# channel 0 (left)
sg.sine(channel=0, freq=1000*1, scale=1.41/1)  # square wave harmonic 1
sg.sine(channel=0, freq=1000*3, scale=1.41/3)  # square wave harmonic 3
sg.sine(channel=0, freq=1000*5, scale=1.41/5)  # square wave harmonic 5
sg.sine(channel=0, freq=1000*7, scale=1.41/7)  # square wave harmonic 7

# channel 1 (right)
sg.square(channel=1, freq=1000)  # ideal square wave

sg.plot()
sg.play(10)

In [ ]:
sg.stop()

**Sweep a frequency range**

In [ ]:
from vds1022.generator import Generator

sg = Generator(device=None, scale=1)
sg.sweep(duration=1, f0=1, f1=1000)
sg.play(10)

In [ ]:
sg.stop()